In [8]:
from IPython.core.display import display
import pandas as pd
import numpy as np
from pykakasi import kakasi
from sklearn.pipeline import Pipeline
from catboost import Pool,CatBoostRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
df = pd.read_csv('./date/train.csv',index_col=['datetime'],header=0)
dfs = pd.read_csv('./date/test.csv',index_col=['datetime'],header=0)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)

In [3]:
df['payday'] = df['payday'].fillna(0)
df['precipitation'] = df['precipitation'].replace('--', '0')
df['kcal'] = df['kcal'].fillna(404.409639)

dfs['payday'] = dfs['payday'].fillna(0)
dfs['precipitation'] = dfs['precipitation'].replace('--', '0')
dfs['kcal'] = dfs['kcal'].fillna(404.409639)

df = df.astype({'soldout': 'object', 'payday': 'object'})
dfs = dfs.astype({'soldout': 'object', 'payday': 'object'})

In [4]:
df['days'] = range(1, 1+df.shape[0])
print(df.shape)
df

(207, 12)


,y,week,soldout,name,kcal,remarks,event,payday,weather,precipitation,temperature,days
datetime,,,,,,,,,,,,
2013-11-18,90,月,0,厚切りイカフライ,404.409639,NaN,NaN,0,快晴,0,19.8,1
2013-11-19,101,火,1,手作りヒレカツ,404.409639,NaN,NaN,0,快晴,0,17.0,2
2013-11-20,118,水,0,白身魚唐揚げ野菜あん,404.409639,NaN,NaN,0,快晴,0,15.5,3
2013-11-21,120,木,1,若鶏ピリ辛焼,404.409639,NaN,NaN,0,快晴,0,15.2,4
2013-11-22,130,金,1,ビッグメンチカツ,404.409639,NaN,NaN,0,快晴,0,16.1,5
2013-11-25,135,月,1,鶏の唐揚,404.409639,NaN,NaN,0,曇,0,14.6,6
2013-11-26,145,火,0,豚のスタミナ炒め,404.409639,NaN,NaN,0,快晴,0,17.9,7
2013-11-27,140,水,1,ボローニャ風カツ,404.409639,NaN,NaN,0,晴れ,0,14.7,8
2013-11-28,151,木,0,ハンバーグ,404.409639,NaN,NaN,0,薄曇,0,17.7,9


In [5]:
dfs['days'] = range(208, 208+dfs.shape[0])
print(dfs.shape)
dfs

(40, 11)


,week,soldout,name,kcal,remarks,event,payday,weather,precipitation,temperature,days
datetime,,,,,,,,,,,
2014-10-1,水,1,メンチカツ,420.000000,NaN,NaN,0,雨,0,20.2,208
2014-10-2,木,0,バーベキューチキン,415.000000,NaN,NaN,0,曇,0,23.9,209
2014-10-3,金,0,豚肉のマスタード焼き,405.000000,NaN,NaN,0,晴れ,0,28.7,210
2014-10-6,月,1,麻婆春雨,400.000000,NaN,NaN,0,雨,0.5,21.5,211
2014-10-7,火,0,厚揚げ肉みそ炒め,430.000000,NaN,NaN,0,晴れ,0,22.1,212
2014-10-8,水,0,完熟トマトのホットカレー,420.000000,NaN,NaN,0,晴れ,0,23.3,213
2014-10-9,木,0,豚キムチ炒め,435.000000,NaN,キャリアアップ支援セミナー,0,曇,0,22.5,214
2014-10-10,金,0,ポークカレー,404.409639,お楽しみメニュー,NaN,1,薄曇,0,26.1,215
2014-10-14,火,0,若鶏梅肉包揚げ,408.000000,NaN,NaN,0,快晴,0,26.8,216


In [6]:
text = df['name']
kakasi = kakasi()  # Generate kakasi instance

kakasi.setMode("H", "a")  # Hiragana to ascii
kakasi.setMode("K", "a")  # Katakana to ascii
kakasi.setMode("J", "a")  # Japanese(kanji) to ascii

kakasi.setMode("r", "Hepburn")  # Use Hepburn romanization

conv = kakasi.getConverter()
result = conv.do(text)
print(result)

datetime
2013-11-18            atsushikiriikafurai
2013-11-19             tezukurihirekatsu
2013-11-20          shiromizakanakaraageyasaian
2013-11-21              wakadoripirishinyaki
2013-11-22            biggumenchikatsu
2013-11-25                niwatorinotouyou
2013-11-26            butanosutaminaitame
2013-11-27            boroonyakazekatsu
2013-11-28               hanbaagu
2013-11-29       tarutaruinsooseejikatsu
2013-12-2                maabotoufu
2013-12-3             atsushiagebutashougaitame
2013-12-4         kuriimuchiizuirimenchi
2013-12-5          niwatorinokasshunattsuitame
2013-12-6             tezukuriroosukatsu
2013-12-9           hanbaagudemisoosu
2013-12-10       yawarakaroosunosamujon
2013-12-11                gomokugohan
2013-12-12                nikujaga
2013-12-13            tandoriichikin
2013-12-16           kakifuraitarutaru
2013-12-17             tezukurihirekatsu
2013-12-18                 hoikourou
2013-12-19             pookumisoyaki
2013-12-20           

In [9]:
text2 = dfs['name']
kakasi = kakasi()  # Generate kakasi instance

kakasi.setMode("H", "a")  # Hiragana to ascii
kakasi.setMode("K", "a")  # Katakana to ascii
kakasi.setMode("J", "a")  # Japanese(kanji) to ascii

kakasi.setMode("r", "Hepburn")  # Use Hepburn romanization

conv = kakasi.getConverter()
result2 = conv.do(text2)
print(result2)

datetime
2014-10-1              menchikatsu
2014-10-2          baabekyuuchikin
2014-10-3         butanikunomasutaadoyaki
2014-10-6               asabaaharusame
2014-10-7           atsushiagenikumisoitame
2014-10-8       kanjukutomatonohottokaree
2014-10-9             butakimuchiitame
2014-10-10            pookukaree
2014-10-14           wakadoriumenikutsutsumiage
2014-10-15           mikkusuguriru
2014-10-16        butanikutohakusainochuukaitame
2014-10-17              hirekatsu
2014-10-20               butayanagawa
2014-10-21           pookushougayaki
2014-10-22              maaboudoufu
2014-10-23         biifusutoroganofu
2014-10-24              karaagedonburi
2014-10-27             masushioyakiki
2014-10-28          chinjaoroosu
2014-10-29           atsushisetsuikafurai
2014-10-30       torinikutoyasainokurokoshouitame
2014-10-31               happousai
2014-11-4            butanikunoshougayaki
2014-11-5              hanbaagu
2014-11-6       irodoriyasaitotorinikunokurosuan
2014-11-

In [10]:
df['name'] =     ( ['atsushikiriikafurai'],
                   ['tezukurihirekatsu'],
                   ['shiromizakanakaraageyasaian'],
                   ['wakadoripirishinyaki'],  
                   ['biggumenchikatsu'],
                   ['niwatorinotouyou'],
                   ['butanosutaminaitame'],
                   ['boroonyakazekatsu'],
                   ['hanbaagu'],
                   ['tarutaruinsooseejikatsu'],
                   ['maabotoufu'],
                   ['atsushiagebutashougaitame'],
                   ['kuriimuchiizuirimenchi'],
                   ['niwatorinokasshunattsuitame'],  
                   ['tezukuriroosukatsu'],
                   ['hanbaagudemisoosu'],
                   ['yawarakaroosunosamujon'],
                   ['gomokugohan'],
                   ['nikujaga'],
                   ['tandoriichikin'],
                   ['kakifuraitarutaru'],
                   ['tezukurihirekatsu'],
                   ['hoikourou'],
                   ['pookumisoyaki'],  
                   ['niwatorinokaraagekansuan'],
                   ['sakkurimenchikatsu'],
                   ['tegonekazehanbaagu'],
                   ['subuta'],
                   ['kareeirisooseejikatsu'],
                   ['butanikunoshougayaki'],
                   ['niwatorichirisoosu'],
                   ['tezukuriroosukatsu'],
                   ['niwatorinoteriyakimasutaado'],
                   ['sanmashinajiyaki'],  
                   ['kareikaraageyasaiankake'],
                   ['hoikourou'],
                   ['juushiimenchikatsu'],
                   ['sabayakimisokake'],
                   ['tezukurihirekatsutokaree'],
                   ['subuta'],
                   ['niwatorinoremonpeppaayakiorkaree'],
                   ['chinjaoroosu'],
                   ['ebifuraitarutaru'],
                   ['chiizuirimenchikatsu'],  
                   ['niwatorinokaraage'],
                   ['medaiteriyaki'],
                   ['hanbaagukareesoosu'],
                   ['sawarayakimisokake'],
                   ['niwatorinopirishinyaki'],
                   ['tezukuriroosukatsu'],
                   ['hotatekuriimushichuu'],
                   ['niwatorinokaraageoroshisoosu'],
                   ['masunomasutaadosoosu'],
                   ['hoikourou'],  
                   ['roosuamamisoyaki'],
                   ['chinjaoroosu'],
                   ['ebifuraitohotatekushikatsu'],
                   ['hanbaaguwafuukinokosoosu'],
                   ['tandoriichikin'],
                   ['subutaorkaree'],
                   ['pookuhayashi'],
                   ['maabotoufu'],
                   ['niwatorichirisoosu'],
                   ['shiromizakanakaraageyasaiankake'],  
                   ['tezukurihirekatsu'],
                   ['hanbaagukareesoosu'],
                   ['hoikourou'],
                   ['menchikatsu'],
                   ['chikinkuriimushichuu'],
                   ['boroonyakazekatsu'],
                   ['ebikuriimiikunooderu'],
                   ['nikujaga'],
                   ['biifukaree'],
                   ['tezukuriroosukatsu'],  
                   ['kareiyasaiankake'],
                   ['niwatorinokaraageoroshisoosu'],
                   ['chiizuirihanbaagu'],
                   ['ettoukyabetsunomenchikatsu'],
                   ['niwatorinooyakoni'],
                   ['maabotoufu'],
                   ['tezukurihirekatsu'],
                   ['nikudangokuriimushichuu'],
                   ['roosuamamisoyaki'],
                   ['kiimakaree'],  
                   ['aoshounikuito'],
                   ['menchikatsu'],
                   ['nikujaga'],
                   ['subuta'],
                   ['wafuusoosuhanbaagu'],
                   ['aokyounagyuunikuitame'],
                   ['nikudangonoshichuu'],
                   ['aoshounikuito'],
                   ['niwatorinokaraageoroshisoosu'],
                   ['chikinkaree'],  
                   ['biifutomatoni'],
                   ['pookushougayaki'],
                   ['kareikaraageyasaiankake'],
                   ['gyuudonkazeni'],
                   ['niwatorinomisotsukeyaki'],
                   ['gyuunikutakenokoni'],
                   ['maabotoufu'],
                   ['tezukurihirekatsu'],
                   ['niwatorinoteriyakimayo'],
                   ['chuukadon'],  
                   ['tandoriichikin'],
                   ['butamisomenchikatsu'],
                   ['maabonasu'],
                   ['niwatorinotenpura'],
                   ['nikujaga'],
                   ['menchikatsu'],
                   ['tezukurichikinkatsu'],
                   ['kinokosoosuhanbaagu'],
                   ['shiromizakanakaraageyasaiankake'],
                   ['menchikatsu'],  
                   ['tandoriichikin'],
                   ['pookukaree'],
                   ['butanikutonasunopirishinitame'],
                   ['niwatorinomisotsukeyaki'],
                   ['chiizuhanbaagu'],
                   ['saamonnomunieru2shokusoosu'],
                   ['gyuunikukorokke'],
                   ['niwatorinokasshunattsuitame'],
                   ['gyuunikusukiyakikaze'],
                   ['niwatorinokaraageoroshisoosu'],  
                   ['ikatenpura'],
                   ['chikinkaree'],
                   ['hanbaaguketchappusoosu'],
                   ['gooyachanpuruu'],
                   ['tappuribeekonfurai'],
                   ['niwatorinomisotsukeyaki'],
                   ['tezukuriroosukatsu'],
                   ['gyuunikuninnikunomeitame'],
                   ['kareikaraageyasaiankake'],
                   ['chapuche'],  
                   ['tandoriichikin'],
                   ['kiimakaree'],
                   ['ushisukiyakikaze'],
                   ['pookusoteekankokusoosu'],
                   ['biifusutoroganofu'],
                   ['ajikaraagenanbansoosu'],
                   ['takikomigomeshi'],
                   ['maabonasu'],
                   ['niwatorinotomatoshichuu'],
                   ['butanohiyashiyabu'],  
                   ['kisutoyasainotenpura'],
                   ['gyuudon'],
                   ['niwatorinoshiokaraage'],
                   ['kareikaraagenatsuyasaian'],
                   ['tandoriichikin'],
                   ['shiromizakanamunieru'],
                   ['tezukuritonkatsu'],
                   ['wafuuhanbaagu'],
                   ['kajikinokanshinni'],
                   ['chikinnokoonkuriimuyaki'],  
                   ['purukogi'],
                   ['pookukaree'],
                   ['niwatorinokaraagenegishioitamesoosu'],
                   ['butareishabuyasaizoe'],
                   ['shiromizakanafurai'],
                   ['butasukiyaki'],
                   ['ebifurai'],
                   ['chinjaoroosu'],
                   ['happousai'],
                   ['janbochikinkatsu'],  
                   ['hiyashitanukiudon.takikomigomeshi'],
                   ['butanikunomasutaadoyaki'],
                   ['baabekyuuchikin'],
                   ['menchikatsu'],
                   ['niwatorinokaraagesuiitochirisoosu'],
                   ['butanikunoshougayaki'],
                   ['hanbaagu（demikinokobataa）'],
                   ['torinikunokareetouyou'],
                   ['butakimuchiitame'],
                   ['chikinkaorikusayakimasutaadosoosu'],  
                   ['chikinkaree'],
                   ['saamonfurai.tarutaru'],
                   ['atsushisetsuhamukatsu'],
                   ['youshokuyasannomenchikatsu'],
                   ['ushisukiyaki'],
                   ['butaroosunopizakazechiizuyaki'],
                   ['chikinnanban'],
                   ['rokomokodonburi'],
                   ['shiromizakananonanbuyaki'],
                   ['kiimakaree'],  
                   ['kareinokaraage'],
                   ['butanikunogomashabu'],
                   ['chikinnokaramisoyaki'],
                   ['biifushichuu'],
                   ['nagoyamisokatsu'],
                   ['oyakoni'],
                   ['chikinsuteeki.kinokosoosu'],
                   ['menchikatsu'],
                   ['torinikunosanzokuyaki'],
                   ['hanbaagudemisoosu'],  
                   ['buriremonpeppaayaki'],
                   ['tezukurichikinkatsu'],
                   ['pookukaree'],
                   ['chiizumenchikatsu'],
                   ['chikinfurikasse'],
                   ['kareikaraage kansuan'],
                   ['atsushisetsuikafurai'],
                   ['chikuzenni'],
                   ['shiromizakananomasutaadoyaki'],
                   ['ushikarubiyakiniku'],  
                   ['ranchibyuffe'],
                   ['butanikutotamagonoitame'],
                   ['torinikutokashuunattsuitame'])

df

,y,week,soldout,name,kcal,remarks,event,payday,weather,precipitation,temperature,days
datetime,,,,,,,,,,,,
2013-11-18,90,月,0,"(atsushikiriikafurai,)",404.409639,NaN,NaN,0,快晴,0,19.8,1
2013-11-19,101,火,1,"(tezukurihirekatsu,)",404.409639,NaN,NaN,0,快晴,0,17.0,2
2013-11-20,118,水,0,"(shiromizakanakaraageyasaian,)",404.409639,NaN,NaN,0,快晴,0,15.5,3
2013-11-21,120,木,1,"(wakadoripirishinyaki,)",404.409639,NaN,NaN,0,快晴,0,15.2,4
2013-11-22,130,金,1,"(biggumenchikatsu,)",404.409639,NaN,NaN,0,快晴,0,16.1,5
2013-11-25,135,月,1,"(niwatorinotouyou,)",404.409639,NaN,NaN,0,曇,0,14.6,6
2013-11-26,145,火,0,"(butanosutaminaitame,)",404.409639,NaN,NaN,0,快晴,0,17.9,7
2013-11-27,140,水,1,"(boroonyakazekatsu,)",404.409639,NaN,NaN,0,晴れ,0,14.7,8
2013-11-28,151,木,0,"(hanbaagu,)",404.409639,NaN,NaN,0,薄曇,0,17.7,9


In [11]:
y = df.iloc[:,0]
X = df.iloc[:,1:]
print(X.shape)
print(y.shape)

(207, 11)
(207,)


In [12]:
ohe_columns = ['week',
               'remarks',
               'event',
               'weather']

X_new = pd.get_dummies(X,
                       dummy_na=True,
                        
                       columns=ohe_columns)


X_new_columns = X_new.columns.values

print(X_new.shape)
pd.set_option('display.max_columns', 50)
X_new.head(50)

(207, 31)


,soldout,name,kcal,payday,precipitation,temperature,days,week_月,week_木,week_水,week_火,week_金,week_nan,remarks_お楽しみメニュー,remarks_スペシャルメニュー（800円）,remarks_手作りの味,remarks_料理長のこだわりメニュー,remarks_酢豚（28食）、カレー（85食）,remarks_鶏のレモンペッパー焼（50食）、カレー（42食）,remarks_nan,event_キャリアアップ支援セミナー,event_ママの会,event_nan,weather_快晴,weather_晴れ,weather_曇,weather_薄曇,weather_雨,weather_雪,weather_雷電,weather_nan
datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2013-11-18,0,"(atsushikiriikafurai,)",404.409639,0,0,19.8,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0
2013-11-19,1,"(tezukurihirekatsu,)",404.409639,0,0,17.0,2,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0
2013-11-20,0,"(shiromizakanakaraageyasaian,)",404.409639,0,0,15.5,3,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0
2013-11-21,1,"(wakadoripirishinyaki,)",404.409639,0,0,15.2,4,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0
2013-11-22,1,"(biggumenchikatsu,)",404.409639,0,0,16.1,5,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0
2013-11-25,1,"(niwatorinotouyou,)",404.409639,0,0,14.6,6,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0
2013-11-26,0,"(butanosutaminaitame,)",404.409639,0,0,17.9,7,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0
2013-11-27,1,"(boroonyakazekatsu,)",404.409639,0,0,14.7,8,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0
2013-11-28,0,"(hanbaagu,)",404.409639,0,0,17.7,9,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0


In [20]:
from sklearn.pipeline import Pipeline
from catboost import Pool,CatBoostRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


list_cat_features = ['soldout', 'kcal', 'payday', 'precipitation', 'temperature', 'days','week_月', 'week_木', 'week_水', 'week_火', 'week_金'
                     , 'week_nan', 'remarks_お楽しみメニュー', 'remarks_スペシャルメニュー（800円）', 'remarks_手作りの味', 
                     'remarks_料理長のこだわりメニュー', 'remarks_酢豚（28食）、カレー（85食）',
                     'remarks_鶏のレモンペッパー焼（50食）、カレー（42食）', 'remarks_nan', 'event_キャリアアップ支援セミナー', 
                     'event_ママの会', 'event_nan', 'weather_快晴', 'weather_晴れ', 'weather_曇', 'weather_薄曇', 'weather_雨', 'weather_雪'
                     , 'weather_雷電', 'weather_nan']
list_text_features = ['name']

train_pool = Pool(
    X_new, 
    y, 
    cat_features = list_cat_features,
    text_features = list_text_features,
    feature_names=list(X_new))

pipe_cat = Pipeline([('scl',StandardScaler()),('pca', PCA()),('est',CatBoostRegressor(random_state=0))])
pipe_cat.fit(train_pool)
print(np.average(cross_val_score(pipe_cat,X_new,y,scoring='r2',cv=5)))

CatBoostError: Invalid type for text_feature[non-default value idx=0,feature_idx=1]=('atsushikiriikafurai',) : text_features must have string type